## Manchester AirBnb Assignment
_By Nick Brooks, January 2020_

****Column Explanations****:**
- **Property ID:** Airbnb Property ID
- **Property Type:** Actual property type of the property being listed on Airbnb (Detached house, Flat etc)
- **Listing Type:** The listing choice selected between Entire home, Apartment and Private room Bedrooms:** Number of bedrooms
- **Reporting Month:** The month corresponding to the provided data
- **Occupancy Rate:** Occupancy rate achieved in the given month
- **Revenue (USD):** Revenue achieved in the given month (USD)
- **Revenue (Native):** Revenue achieved in the given month (In currency of booking)
- **ADR (USD):** Average daily revenue achieved in the given month (USD)
- **ADR (Native):** Average daily revenue achieved in the given month (In currency of booking)
- **Number of Reservations:** Number of reservations made for the listing in the given month
- **Reservation Days:** Number of days listing was booked in the month
- **Available Days:** Number of days listing was not booked in the month
- **Blocked Days:** Number of days listing was blocked by the host in the month
- **Active:** Whether property is active on Airbnb in the month or not
- **Currency Native:** Currency of the booking being made
- **Host ID:** Host identifier

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import time
notebookstart= time.time()

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict, KFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

import math
import gc

%matplotlib inline
sns.set_style("whitegrid")

pd.options.display.max_rows = 999
pd.options.display.width = 300
pd.options.display.max_columns = 500

plot_title_size = 16

In [ ]:
def plot_cloud(string, ax, title = "WordCloud", cmap = "plasma"):
    wordcloud = WordCloud(width=800, height=500,
                          collocations=True,
                          background_color="white",
                          colormap=cmap
                          ).generate(string)

    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(title,  fontsize=18)
    ax.axis('off')

def big_word_cloud(plot_df, plt_set, columns, figsize, cmap = "plasma"):
    """
    Iteratively Plot WordClouds
    """
    rows = math.ceil(len(plt_set)/columns)
    n_plots = rows*columns
    f,ax = plt.subplots(rows, columns, figsize = figsize)
    for i in range(0,n_plots):
        ax = plt.subplot(rows, columns, i+1)
        if i < len(plt_set):
            str_col = plt_set[i]
            string = " ".join(plot_df.loc[plot_df[str_col].notnull(),str_col].astype(str).str.lower().str.replace("none", "").str.title())
            string += 'EMPTY'
            ax = plt.subplot(rows, 2, i+1)
            plot_cloud(string, ax, title = "{} - {} Missing".format(str_col.title(), plot_df[str_col].isnull().sum()), cmap = cmap)
        else:
            ax.axis('off')
    plt.tight_layout(pad=0)

def big_count_plotter(plot_df, plt_set, columns, figsize, custom_palette = sns.color_palette("Dark2", 15), top_n = 15):
    """
    Iteratively Plot all categorical columns
    Has category pre-processing - remove whitespace, lower, title, and takes first 30 characters.
    """
    rows = math.ceil(len(plt_set)/columns)
    n_plots = rows*columns
    f,ax = plt.subplots(rows, columns, figsize = figsize)
    for i in range(0,n_plots):
        ax = plt.subplot(rows, columns, i+1)
        if i < len(plt_set):
            c_col = plt_set[i]
            plt_tmp = plot_df.loc[plot_df[c_col].notnull(),c_col].astype(str).str.lower().str.strip().str.title().apply(lambda x: x[:30])
            plot_order = plt_tmp.value_counts().index[:top_n]
            sns.countplot(y = plt_tmp, ax = ax, order = plot_order, palette = custom_palette)
            ax.set_title("{} - {} Missing".format(c_col.title(), plot_df[c_col].isnull().sum()), fontsize = plot_title_size)
            ax.set_ylabel("{} Categories".format(c_col.title()))
            ax.set_xlabel("Count")
        else:
            ax.axis('off')

    plt.tight_layout(pad=1)
    
def big_boxplotter(plot_df, plt_set, columns, figsize, custom_palette = sns.color_palette("Dark2", 15), quantile = .99):
    rows = math.ceil(len(plt_set)/columns)
    n_plots = rows*columns
    f,ax = plt.subplots(rows, columns, figsize = figsize)
    palette = itertools.cycle(custom_palette)
    for i in range(0,n_plots):
        ax = plt.subplot(rows, columns, i+1)
        if i < len(plt_set):
            cont_col = plt_set[i]
            plt_tmp = plot_df.loc[plot_df[cont_col].notnull(),cont_col].astype(float)
            col_max = plt_tmp.max()
            if quantile:
                plt_tmp = plt_tmp[plt_tmp < plt_tmp.quantile(quantile)]
            sns.boxplot(plt_tmp, color = next(palette))
            ax.set_title("{}\n{} Missing - {} Max".format(cont_col.title(), plot_df[cont_col].isnull().sum(), col_max),
                        fontsize = plot_title_size)
            ax.set_xlabel("Values (Upper Limit is {}th Quantile)".format(str(quantile)))
            ax.set_ylabel("{} Boxplot".format(cont_col))
        else:
            ax.axis('off')

    plt.tight_layout(pad=1)
    
def big_ts_count_plotter(plot_df, plt_set, columns, ts_var, figsize, custom_palette = sns.color_palette("Dark2", 15), top_n = 5):
    """
    Iteratively Plot all categorical columns over time
    """
    rows = math.ceil(len(plt_set)/columns)
    n_plots = rows*columns
    f,ax = plt.subplots(rows, columns, figsize = figsize)
    for i in range(0,n_plots):
        ax = plt.subplot(rows, columns, i+1)
        if i < len(plt_set):
            tmp_var = plt_set[i]
            plot_df.loc[plot_df[ts_var].notnull(),:].set_index(ts_var)['placeholder'].resample('1m').count().plot(ax=ax, label="Full Count")
            for tv in plot_df[tmp_var].dropna().value_counts().index[:top_n]:
                plot_df.loc[(plot_df[ts_var].notnull()) & (plot_df[tmp_var] == tv),:]\
                    .set_index(ts_var)['placeholder'].resample('1m').count().plot(ax=ax, label=tv)
            ax.legend(fontsize='large', loc='center left', bbox_to_anchor=(1, 0.5))
            ax.set_title("Monthly Count of {} Variable Through Time".format(tmp_var), fontsize = plot_title_size)
        else:
            ax.axis('off')

    plt.tight_layout(pad=1)

    
def big_ts_mean_plotter(plot_df, plt_set, columns, ts_var, std_multiplier, figsize, custom_palette = sns.color_palette("Dark2", 15)):
    """
    Iteratively Plot all continuous columns over time
    """
    rows = math.ceil(len(plt_set)/columns)
    n_plots = rows*columns
    f,ax = plt.subplots(rows, columns, figsize = figsize)
    palette = itertools.cycle(custom_palette)
    
    for i in range(0,n_plots):
        ax = plt.subplot(rows, columns, i+1)
        if i < len(plt_set):
            c = next(palette)
            tmp_var = plt_set[i]
            monthly_aggs = (plot_df
                           .set_index(ts_var)
                           .resample('1m')
                           .agg({tmp_var: ['mean','std']}))

            monthly_aggs.columns = pd.Index([e[0] +"_"+ e[1] for e in monthly_aggs.columns.tolist()])

            monthly_aggs = monthly_aggs.assign(
                upper = monthly_aggs[tmp_var + "_mean"] + (monthly_aggs[tmp_var + "_std"] * std_multiplier),
                lower = monthly_aggs[tmp_var + "_mean"] - (monthly_aggs[tmp_var + "_std"] * std_multiplier)
            )

            monthly_aggs[tmp_var+'_mean'].plot(ax=ax, color=c, label="{} Average".format(tmp_var))
            ax.fill_between(monthly_aggs.index, monthly_aggs.lower, monthly_aggs.upper, color=c, alpha=0.2)
            ax.legend(fontsize='large', loc='upper center', bbox_to_anchor=(0.5, -0.12))
            ax.set_title("Monthly Count of {} Variable Through Time".format(tmp_var), fontsize = plot_title_size)
        else:
            ax.axis('off')

    plt.tight_layout(pad=1)

def prepare_time_features(df, ts_var):
#     df['hour_of_day'] = df[ts_var].dt.hour
#     df['week'] = df[ts_var].dt.week
    df['month'] = df[ts_var].dt.month
    df["year"] = df[ts_var].dt.year
#     df['day_of_year'] = df[ts_var].dt.dayofyear
    df['week_of_year'] = df[ts_var].dt.weekofyear
#     df["weekday"] = df[ts_var].dt.weekday
    df["quarter"] = df[ts_var].dt.quarter
#     df["day_of_month"] = df[ts_var].dt.day
    
    return df

def time_slicer(plot_df, timeframes, value, agg_method, color="purple", figsize=[12,12]):
    """
    Function to count observation occurrence through different lenses of time.
    """
    f, ax = plt.subplots(len(timeframes), figsize = figsize)
    for i,x in enumerate(timeframes):
        plot_df.loc[:,[x,value]].groupby([x]).agg({value:agg_method}).plot(ax=ax[i],color=color)
        ax[i].set_ylabel(value.replace("_", " ").title())
        ax[i].set_title("{} by {}".format(value.replace("_", " ").title(), x.replace("_", " ").title()))
        ax[i].set_xlabel("")
        ax[i].set_ylim(0,)
    ax[len(timeframes)-1].set_xlabel("Time Frame")
    plt.tight_layout(pad=0)

#### Load Data
- Enforce data types
- Create simple time features

In [ ]:
df = pd.read_csv("/kaggle/input/housing-manchester/Airbnb_Manchester.csv")
print('DF shape: {} rows, {} columns'.format(*df.shape))

print("Number of unique Properties: {}".format(df['Property ID'].nunique()))
print("Number of unique Host: {}".format(df['Host ID'].nunique()))

display(df.sample(5))

#### First impressions:
- Panel Data, monthly time intervals. Between 2014 and 2019, which would very much reflect the dawn and boom of AirBnb.
- Not much information about the actual housing products. 
- Operational features around revenue and occupancy.
- GPS may be powerful feature to understand trendy neighbourhoods.

TLDR - Fairly simple dataset with a decent number of observations (~200k)

## Data Cleaning and Exploration (40%)
- Clean the dataset of unwanted data and outliers
- Deal with missing data values

### Build Features:
- How long Host / Property has been in action
- Buckets for how enterpricing the Host is. (Owned properties)
- Distance from Manchester City Center
- Time features

In [ ]:
df['Reporting Month'] = pd.to_datetime(df['Reporting Month'])
df = prepare_time_features(df, ts_var = 'Reporting Month')
df['placeholder'] = 1

In [ ]:
# Get the last active date of property
df = pd.merge(df,
         df.loc[df.Active == True,:].groupby('Property ID')['Reporting Month'].agg('max').rename("property_last_active").reset_index(),
         how='left', on = 'Property ID')
# Get the first active date of property
df = pd.merge(df,
         df.loc[df.Active == True,:].groupby('Property ID')['Reporting Month'].agg('min').rename("property_first_active").reset_index(),
         how='left', on = 'Property ID')
# Get the first active date of property
df = pd.merge(df,
         df.loc[df.Active == True,:].groupby('Host ID')['Reporting Month'].agg('max').rename("host_last_active").reset_index(),
         how='left', on = 'Host ID')
df = pd.merge(df,
         df.loc[df.Active == True,:].groupby('Host ID')['Reporting Month'].agg('min').rename("host_first_active").reset_index(),
         how='left', on = 'Host ID')

# Create features for Host and Properties around activity over time
df = df.assign(
    # Since First Active in days
    days_since_properties_first_active = (df['Reporting Month'] - df['property_first_active']).dt.days,
    days_since_host_first_active = (df['Reporting Month'] - df['host_first_active']).dt.days,
    # Until last Active in days
    days_until_properties_last_active = (df['property_last_active'] - df['Reporting Month']).dt.days,
    days_until_host_last_active = (df['host_last_active'] - df['Reporting Month']).dt.days,
    # Total life in days
    total_days_property_active = (df['property_last_active'] - df['property_first_active']).dt.days,
    total_days_host_active = (df['host_last_active'] - df['host_first_active']).dt.days,
    # Host Related Features
    host_owned_properties = df.groupby('Host ID')["Property ID"].transform('nunique')
)

## Filter Table
- I have observed that around 40% of the dataset has non-active houses at a given month. These cases are void of revenue information, and do not contribute additional information. In fact, they provide a risk of misguiding the analysis.
- The Country, State, City, Currency only have a single factor suiting Manchester. ZipCode is 100% Null. This will be ignored.
- I will frame this analysis in terms of pounds, because this currency is local and will ignore exchange rates.

In [ ]:
print(df.Active.value_counts(normalize=True))

In [ ]:
drop_cols =  [
    'Revenue (USD)',
    'ADR (USD)',
    'Country',
    'State',
    'City',
    'Zipcode',
    'Currency Native'
]

df = df.loc[df.Active == True,:].drop(drop_cols, axis=1)
print('DF shape: {} rows, {} columns'.format(*df.shape))

In [ ]:
df.sample(5)

#### Quick EDA

In [ ]:
sns.boxplot(df.groupby('Host ID')["Property ID"].agg('nunique'))
plt.title("Uniqued Properties owned by HostID")
plt.show()

In [ ]:
count_plot_cols = [
    'Property Type',
    'Listing Type',
    'Bedrooms'
]

big_count_plotter(plot_df = df,
                  plt_set = count_plot_cols,
                  columns = 2,
                  figsize = [15,10])

#### Main takeaways
- House type is dominated by single bedroom Apartments or Houses.

In [ ]:
big_ts_count_plotter(plot_df = df.loc[df.Active == True,:],
                      plt_set = count_plot_cols,
                      columns = 2,
                      ts_var = 'Reporting Month',
                      figsize = [20,10])

#### Main takeaways:
- Appears to be a linear growth rate. This finding should be validated against unique properties.
- Without the time-element, the top factors in categories appear more unequal than the reality. This is because the total difference is the sum of minor difference across (4 years * 12 months) ~48 montly periods

In [ ]:
f, ax = plt.subplots(1,2, figsize = [14,5])
tmp_plot = df.groupby('year').agg({"Host ID": "nunique", "Property ID": "nunique"})
tmp_plot.plot(ax=ax[0])
ax[0].set_ylim(0, )
ax[0].set_title("Number of Unique Host and Properties", fontsize = plot_title_size)

tmp_plot['Properties to Host Ratio'] = tmp_plot['Property ID'] / tmp_plot['Host ID']
tmp_plot['Properties to Host Ratio'].plot(ax=ax[1])
ax[1].set_ylim(1, 2)
ax[1].set_title("Properties to Host Ratio", fontsize = plot_title_size)
ax[1].set_ylabel("Properties to Host Ratio")
plt.tight_layout(pad=2)
plt.show()

#### Main takeaways: <br>
- Over the last 4 years, there has been a minor increase in the average number of properties held by a host.
- 2019 suggests a dip because the year is not complete yet.

In [ ]:
continuous_cols = [
    'Occupancy Rate',
    'Revenue (Native)',
    'ADR (Native)',
    'Number of Reservations',
    'Reservation Days',
    'Available Days',
    'Blocked Days',
    'days_since_properties_first_active',
    'days_since_host_first_active',
    'days_until_properties_last_active',
    'days_until_host_last_active',
    'total_days_property_active',
    'total_days_host_active'
]

big_boxplotter(plot_df = df,
               plt_set = continuous_cols,
               columns = 3,
               figsize = [20,25],
               quantile = .98)

#### Main takeaways: <br>
- Host have been on the platform for an average of 2 years.
- A property is available for 15 days on average a month, but also has a very wide variance.
- A property is blocked for around 3 days on average a month.
- Occupancy rate is around 35% on average
- The revenue columns seem to have a outlier, perhaps a handful of mega-profitable houses. These should be double checked..

In [ ]:
df.loc[df['Revenue (Native)'] > 4000,:].sort_values(by='Revenue (Native)', ascending=False).iloc[:10]

I just looks like there was an expensive Condominium that was rented out for 20 days at 7.8K pounds a day!

In [ ]:
continuous_cols = [
    'Occupancy Rate',
    'Revenue (Native)',
    'ADR (Native)',
    'Number of Reservations',
    'Reservation Days',
    'Available Days',
    'Blocked Days',
    'days_since_properties_first_active',
    'days_since_host_first_active',
    'days_until_properties_last_active',
    'days_until_host_last_active',
    'total_days_property_active',
    'total_days_host_active'
]

f, ax = plt.subplots(figsize = [8,5])
sns.heatmap(df.loc[:,continuous_cols].corr(method = 'spearman'),
            annot=False, fmt=".2f",cbar_kws={'label': 'Correlation Coefficient'},
            cmap="coolwarm",ax=ax, linewidths=.3, vmin=-1, vmax=1)
ax.set_title("Continuous Variables Correlation Matrix", fontsize = 14)
plt.show()

#### Main takeaways: <br>
- Using Spearman rank for more robust (non-parametric) correlation test.
- The revenue and availability features are highly correlated (Share data-generation process)
- Days columns are highly correlated (Share data-generation process).
- I'm surprised to see that the days property has been on the platform does not correlate with more occupancy. I would of thought that there would be a feedback loop where successful AirBnBs are less likely to drop off, and that successful AirBnbs get more comments and attention, hence more rents (network effect)

In [ ]:
upper_quantile = .90
lower_qunatile = .10

sns.jointplot(x="Longitude", y="Latitude", data=df.loc[
    (df['Longitude'] < df['Longitude'].quantile(upper_quantile)) &
    (df['Longitude'] > df['Longitude'].quantile(lower_qunatile)) &
    (df['Latitude'] < df['Latitude'].quantile(upper_quantile)) &
    (df['Latitude'] > df['Latitude'].quantile(lower_qunatile))
    ,["Longitude", "Latitude"]], kind="kde", color = 'green')
plt.title("2D Density Plot");

#### Main takeaways: <br>
- There are two epicenter neighbourhoods
- There are also some mild satelite areas in the general south west direction.

In [ ]:
continuous_cols = [
    'Occupancy Rate',
    'Revenue (Native)',
    'ADR (Native)',
    'Number of Reservations',
    'Reservation Days',
    'Available Days',
    'Blocked Days'
]

big_ts_mean_plotter(plot_df = df,
                      plt_set = continuous_cols,
                      columns = 2,
                      std_multiplier = .5,
                      ts_var = 'Reporting Month',
                      figsize = [20,25])

#### Main takeaways: <br>
- There appears to be some seasonal patterns to the occupancy features. The pattern is an annual cycle of slow buildup until a peak in the fall, followed by a stabalizing drop.
- Should followup to see whether there is an annual trend to observe.
- There is a trend of aparments being available for a greater number of days over the years.

In [ ]:
time_slicer(plot_df = df,
            agg_method = 'mean',
            timeframes=['year', 'quarter', "month"],
            value = "Occupancy Rate", color="green", figsize = [10,7])

#### Main takeaways: <br>
- Minor increase in occupancy rate over the years.
- Annual cycle is lowest at .35 during new years, and .55 in september.

In [ ]:
time_slicer(plot_df = df,
            agg_method = 'sum',
            timeframes=['year', 'quarter', "month"],
            value = "Revenue (Native)", color="red", figsize = [10,7])

## Question of Hotness
Now that I have explored the data, I am prepared to determine my dependent variable. My take on hotness is that it is primarily a reflection of what customers deem desirable. For this reason, many aspects of the revenue variables are disqualified, even though an economic argument could be made that the more money people are willing to spend, the more they must like a given property. This line of thinking may be distorted by the fact that ADR (Average Daily Revenue in given month) has an average of 50, but also a significant right tail that leads into the three hundred levels and over.

This leaves the availability related variables. While occupancy rate could have high potential, it may also be distorted by apartments with high blocked days and average reservation days scenarios.

This leaves me between reservation days and number of reservations. This is a tough trade off because on the one hand, reservation days suggests that available and rented apartments will climb to the top of the hotness metric, but it is at risk of scenarios where a single individuals rents an apartment over a long period of time. In this case, the motivations for renting are more likely tied to need for shelter.  <br>
Number of reservations, on the other hand, is representative of customers going through the apartment scoping process and selecting a given apartment X times in a given month. This product therefore most often persuades people to buy it.

## Data Aggregation (10%)
- Aggregate the data by Property ID to create a table of yearly averages of the suitable fields (such as revenue, occupancy etc)
- Convert the dataset that is in the form of monthly incomes to one which is yearly, and only keep data between 2018 and 2019

In [ ]:
columns_for_annual_sum = [
    'Revenue (Native)',
    'Number of Reservations',
    'Reservation Days',
    'Available Days',
    'Blocked Days'
]

static_columns = [
    'Property ID',
    'Property Type',
    'Listing Type',
    'Bedrooms',
    'Latitude',
    'Longitude',
    'year',
    'Active',
    'Host ID',
    'total_days_property_active',
    'total_days_host_active',
    'host_owned_properties']

In [ ]:
model_df = df.loc[df.year == 2018,:].copy()
print('Model DF shape: {} rows, {} columns'.format(*model_df.shape))

agg_df = model_df.groupby(['year','Property ID']).agg({k:'sum' for k in columns_for_annual_sum}).reset_index()
print('agg_df DF shape: {} rows, {} columns'.format(*agg_df.shape))
year_df = pd.merge(
    agg_df,
    model_df.loc[:,static_columns].drop_duplicates(), on = ['year','Property ID'], how = 'left', validate = 'one_to_one')
print('year_df DF shape: {} rows, {} columns'.format(*year_df.shape))
display(year_df.sample(5))
del agg_df, model_df

In [ ]:
# Rebuild lost features
year_df = year_df.assign(
    native_ADR = year_df['Revenue (Native)'] / year_df["Reservation Days"], # Revenue / Reservation days = ADR
    occupancy = year_df['Reservation Days'] / 365
).set_index('Property ID')

year_df['native_ADR'].fillna(0, inplace=True)

In [ ]:
year_df['Reservation Days'].describe()

## Feature Engineering and Exploration (40%)
- Normalise the data and carry out relevant transformations such as Integer encoding or One-Hot encoding if necessary
- Engineer basic features (apart from the ones available) for predicting which properties are considered ‘hot’ (interpret this in your own way)

In [ ]:
dependent_variables  = 'Reservation Days'
categorical_variables = [
    'Property Type',
    'Listing Type'
]
continuous_variables = [
    'native_ADR',
    'host_owned_properties',
    'Bedrooms',
    'Latitude',
    'Longitude',
    'total_days_property_active',
    'total_days_host_active',
]

In [ ]:
def dummy_variables(process_df, model_categorical):
    df = process_df.copy()
    dummy_df = pd.DataFrame()
    for cat_col in model_categorical:
        df[cat_col] = df[cat_col].str.lower().str.strip()
        print("{} Missing Values for {} - Dtype {}".format(df[cat_col].isnull().sum(), cat_col, df[cat_col].dtypes))
        df.loc[~df[cat_col].isin(df[cat_col].value_counts().index[:10]), cat_col] = cat_col + 'Other'
        
        
        dummies = pd.get_dummies(df[cat_col])
        dummies.columns = [x + "_" + cat_col for x in dummies.columns]
        dummy_df = pd.concat([dummy_df, dummies],axis = 1)
        
    return dummy_df

def standardscaler_forpd(process_df, model_continuous):
    df = process_df.loc[:,model_continuous].copy()
    for col in df.columns:
        scaler = StandardScaler()
        df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
        
    return df

In [ ]:
# Dummy variables for categorical..
dummy_df = dummy_variables(process_df = year_df, model_categorical = categorical_variables)
print(dummy_df.shape)

# Standard Scaler for Continuous..
cont_df = standardscaler_forpd(process_df = year_df, model_continuous = continuous_variables)
print(cont_df.shape)
display(cont_df.describe())

X = pd.concat([cont_df, dummy_df], axis =1)
y = year_df[dependent_variables].copy()
print("Dependent Variable Shape",y.shape)
print("Training Set Shape",X.shape)
del dummy_df, cont_df

## Modelling (10%)
- Use these features plus the ones available in the data already and do some basic analysis and construct a Machine Learning model of your choice (whatever you think is suitable) for generating the prediction
- Evaluate the model using appropriate evaluation metrics and explain your choice

In [ ]:
X['intercept'] = 1
model = LinearRegression(fit_intercept = False)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
predicted = cross_validate(model, X, y, cv=kf, scoring = 'neg_mean_absolute_error')
print("CV MAE: {:.4f} +/- {:.4f}".format(abs(predicted['test_score'].mean()), predicted['test_score'].std()))

predicted = cross_val_predict(model, X, y, cv=kf)

# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html#sphx-glr-auto-examples-model-selection-plot-cv-predict-py
fig, ax = plt.subplots()
ax.scatter(y, predicted, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
ax.set_title('Cross Validation Predictions\nAnalyze Model Residuels vs. Actual Out of Fold')
plt.show()

Good to see low fluctuations across folds, however, looking at the prediction output, linear regression is unable to higher values of my dependent variable.

In [ ]:
print("Light Gradient Boosting Regressor: ")
lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    "learning_rate": 0.05,
#     "num_leaves": 180,
#     "feature_fraction": 0.50,
#     "bagging_fraction": 0.50,
#     'bagging_freq': 4,
#     "max_depth": -1,
#     "reg_alpha": 0.3,
#     "reg_lambda": 0.1,
#     "min_split_gain":0.2,
#     "min_child_weight":10,
#     'zero_as_missing':True
                }
lgtrain = lgb.Dataset(X, y ,feature_name = "auto")

lgb_cv = lgb.cv(
    params = lgbm_params,
    train_set = lgtrain,
    num_boost_round=2000,
    stratified=False,
    nfold = 5,
    verbose_eval=50,
    seed = 23,
    early_stopping_rounds=75)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.10, random_state=23)

lgtrain = lgb.Dataset(X_train, y_train)
lgvalid = lgb.Dataset(X_valid, y_valid)

lgb_reg = lgb.train(
    lgbm_params,
    lgtrain,
    num_boost_round=1000,
    valid_sets=[lgtrain, lgvalid],
    valid_names=['train','valid'],
    early_stopping_rounds=50,
    verbose_eval=100)
print("Model Evaluation Stage")
print('Validation Set MAE:', mean_absolute_error(y_valid, lgb_reg.predict(X_valid)))

# Feature Importance Plot
f, ax = plt.subplots(figsize=[7,10])
lgb.plot_importance(lgb_reg, max_num_features=50, ax=ax)
plt.title("Light GBM Feature Importance")

LGBM is able to get a better score of 34 (CV score), which is 40% improvement over regression. It also provides a feature importance plot which helps to double check whether there are un-reasonable features in the mix (leaky).

In [ ]:
print("Notebook Runtime: %0.2f Minutes"%((time.time() - notebookstart)/60))